In [2]:
import os
import pandas as pd
import warnings

from sqlalchemy import create_engine
from sqlalchemy.sql import text
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings('ignore')

### Загрузка Данных

In [4]:
banks_df = pd.read_excel('./scraped_data/banks.xlsx')
offers_df = pd.read_excel('./scraped_data/offers.xlsx')

In [5]:
banks_df.head()

,Unnamed: 0,name,rate,online_rate,term,amount_from,amount_to,offer_count,final_rate,date
0,0,Транскапиталбанк,23.9,0.00,180 дней,100000,3000000.0,3,23.9,2025-01-07 19:20:46.828
1,1,Локо-Банк,23.5,0.00,182 дня,300000,5000000.0,1,23.5,2025-01-07 19:20:46.828
2,2,Банк ДОМ.РФ,22.7,0.00,181 день,10000,15000000.0,6,22.7,2025-01-07 19:20:46.828
3,3,Газпромбанк,24.0,0.01,от 1 дня,1,1500000.0,0,24.0,2025-01-07 19:20:46.828
4,4,Альфа-Банк,20.0,20.00,от 1 дня,1,1500000.0,0,20.0,2025-01-07 19:20:46.828


In [6]:
offers_df.head()

,Unnamed: 0,bank_name,rate,online_rate,term,amount_from,amount_to,final_rate,date
0,0,Транскапиталбанк,23.25,0.0,365 дней,100000,3000000.0,23.25,2025-01-07 19:20:46.829
1,1,Транскапиталбанк,22.20,0.0,90 дней,100000,3000000.0,22.20,2025-01-07 19:20:46.829
2,2,Транскапиталбанк,10.25,0.0,1110 дней,100000,3000000.0,10.25,2025-01-07 19:20:46.829
3,3,Локо-Банк,22.50,0.0,370 дней,300000,5000000.0,22.50,2025-01-07 19:20:46.829
4,4,Банк ДОМ.РФ,22.40,0.0,91 день,10000,15000000.0,22.40,2025-01-07 19:20:46.829


### Подключение к Postgres

In [7]:
# определяем параметры подлчения к postgres
user_name = os.getenv("POSTGRES_DB_USER")
pwd = os.getenv("POSTGRES_DB_PASSWORD")
port = os.getenv("POSTGRES_DB_HOST_PORT")
db_name = os.getenv("POSTGRES_DB_NAME")
host = 'localhost'

# названия таблиц
banks_table_name = 'banks'
offers_table_name = 'offers'

# создаем подключение к postgres
engine = create_engine(f'postgresql://{user_name}:{pwd}@{host}:{port}/{db_name}')

### Создаем Таблицы в БД
Для создание таблицы можно сипользовать несколько способов
- Напрямую вставлять данные
- Используя `pd.io.sql.get_schema`

In [6]:
# способ 
banks_df.to_sql(name=banks_table_name, con=engine, if_exists='replace', index=False)

187

In [7]:
offers_df.to_sql(name=offers_table_name, con=engine, if_exists='replace', index=False)

789

In [48]:
# способ 2
print(pd.io.sql.get_schema(banks_df, name=banks_table_name))

CREATE TABLE "banks" (
"name" TEXT,
  "rate" REAL,
  "online_rate" REAL,
  "term" TEXT,
  "amount_from" INTEGER,
  "amount_to" REAL,
  "offer_count" INTEGER,
  "final_rate" REAL,
  "date" TIMESTAMP
)


In [49]:
# создаем таблицы в БД, если таблицы уже созданые, выведет ошибку, поэтому используем try-except
try:
    with engine.connect() as connection:
        connection.execute(
            # используем text из sqlalchemy, так как он преоразует строку создания таблицы в нужный формат
            # без этого, ругается!
            text(pd.io.sql.get_schema(offers_df, name=offers_table_name, con=engine))
        )
        connection.execute(
            text(pd.io.sql.get_schema(banks_df, name=banks_table_name, con=engine))
        )
except:
    print('Одна из таблиц уже существует! Удали ее!')


Одна из таблиц уже существует! Удали ее!


### Получение Данные из БД

In [50]:
query = """
SELECT
    name,
    rate,
    online_rate,
    term,
    amount_from,
    amount_to,
    offer_count,
    final_rate
FROM
    banks
ORDER BY
    rate DESC
LIMIT 10
"""

# формат исполнения запроса
with engine.connect() as connection:
    result = connection.execute(text(query))
    rows = result.fetchall()
    columns = result.keys()

result_dict = [dict(zip(columns, row)) for row in rows]
result_dict

[{'name': 'ПСБ',
  'rate': 30.0,
  'online_rate': 0.0,
  'term': '91 – 367 дней',
  'amount_from': 50000,
  'amount_to': 50000.0,
  'offer_count': 8,
  'final_rate': 30.0},
 {'name': 'Инвестторгбанк',
  'rate': 25.0,
  'online_rate': 0.0,
  'term': '91 – 370 дней',
  'amount_from': 50000,
  'amount_to': None,
  'offer_count': 6,
  'final_rate': 25.0},
 {'name': 'Банк «Санкт-Петербург»',
  'rate': 25.0,
  'online_rate': 25.0,
  'term': '31 – 1825 дней',
  'amount_from': 10000,
  'amount_to': None,
  'offer_count': 5,
  'final_rate': 25.0},
 {'name': 'Таврический',
  'rate': 25.0,
  'online_rate': 0.0,
  'term': '91 – 1095 дней',
  'amount_from': 50000,
  'amount_to': None,
  'offer_count': 7,
  'final_rate': 25.0},
 {'name': 'БыстроБанк',
  'rate': 25.0,
  'online_rate': 0.0,
  'term': '1830 – 1840 дней',
  'amount_from': 10000,
  'amount_to': 5000000.0,
  'offer_count': 5,
  'final_rate': 25.0},
 {'name': 'Транскапиталбанк',
  'rate': 24.5,
  'online_rate': 0.0,
  'term': '91 – 370 дне

In [51]:
GRAPH_BUTTON_TEXT = "Запросить График 📈"

# ключевая ставка
INTEREST_RATE_BUTTON_TEXT = "Запросить процентную ставку 📊"
INTEREST_RATE_RESPONSE_HEADER = "Вот топ-10 Предложений на Сегодня 💼💰\n"
INTEREST_RATE_RESPONSE_FOOTER = "\n💡Спасибо, что ознакомились с предложениями! Надеемся, эта информация поможет вам принять лучшее решение 🧠💡"

INTEREST_RATE_RESPONSE_STRUCTURE = """\n
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: {name}
Ставка: {rate}%
Онлайн Ставка: {online_rate}%
Срок: {term}
Минимальная Сумма: {amount_from} RUB
Максимальная Сумма: {amount_to} RUB
Количество Предложений: {offer_count}
Итоговая Ставка: {final_rate}%
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
"""

In [52]:
response = INTEREST_RATE_RESPONSE_HEADER
for bank_data in result_dict:
    response += INTEREST_RATE_RESPONSE_STRUCTURE.format(**bank_data)

response += INTEREST_RATE_RESPONSE_FOOTER
print(response)

Вот топ-10 Предложений на Сегодня 💼💰


💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: ПСБ
Ставка: 30.0%
Онлайн Ставка: 0.0%
Срок: 91 – 367 дней
Минимальная Сумма: 50000 RUB
Максимальная Сумма: 50000.0 RUB
Количество Предложений: 8
Итоговая Ставка: 30.0%
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸


💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: Инвестторгбанк
Ставка: 25.0%
Онлайн Ставка: 0.0%
Срок: 91 – 370 дней
Минимальная Сумма: 50000 RUB
Максимальная Сумма: None RUB
Количество Предложений: 6
Итоговая Ставка: 25.0%
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸


💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: Банк «Санкт-Петербург»
Ставка: 25.0%
Онлайн Ставка: 25.0%
Срок: 31 – 1825 дней
Минимальная Сумма: 10000 RUB
Максимальная Сумма: None RUB
Количество Предложений: 5
Итоговая Ставка: 25.0%
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸


💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: Таврический
Ставка: 25.0%
Онлайн Ставка: 0.0%
Срок: 91 – 1095 дней
Минимальная Сумма: 50000 RUB
Максимальная Сумма: None RUB
Количество Предложений: 7
Итоговая Ставка: 25.0%
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸


💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: БыстроБанк
Ставка: 25.0%
Онлайн Ставка: 0.0%
Срок: 1830 – 1840 дней
Минимал